## 1. Rewind your graph

Rewind your graph by fetching a checkpoint using the graph's get_state_history method. You can then resume execution at this previous point in time.

In [1]:
from langchain_openai import ChatOpenAI
import os
llm = ChatOpenAI(
    base_url=os.getenv("LLM_BASE_URL"),
    api_key=os.getenv("LLM_API_KEY"),
    model='qwen-plus-latest',
    streaming=True
)

/Users/daijunjie/miniconda3/envs/langchain-env/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

## 2. Add steps

Add steps to your graph. Every step will be checkpointed in its state history:

In [3]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_6890d74c9e174ffc9a54b0)
 Call ID: call_6890d74c9e174ffc9a54b0
  Args:
    query: LangGraph tutorial and overview
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph tutorial and overview", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.datacamp.com/tutorial/langgraph-tutorial", "title": "LangGraph Tutorial: What Is LangGraph and How to Use It?", "content": "LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph

In [4]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That’s a great idea! Building an autonomous agent with **LangGraph** is a fantastic way to explore its capabilities. You can create agents that **plan, execute tasks, use tools, and even collaborate** with other agents.

Here’s a quick roadmap to help you get started building your autonomous agent:

---

### 🚀 Step-by-Step: Build an Autonomous Agent with LangGraph

#### 1. **Define the Goal**
Decide what your agent should do. For example:
- A **research agent** that answers questions by searching the web and summarizing results.
- A **task manager** that breaks down goals, assigns subtasks, and tracks progress.
- A **customer support bot** that routes issues and escalates when needed.

---

#### 2. **Set Up Your Environment**
Install the required packages:
```b

## 3. Replay the full state history

Now that you have added steps to the chatbot, you can replay the full state history to see everything that occurred.

In [11]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 5:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
-----------------------------------------------

In [12]:
print(to_replay.next)
print(to_replay.config)

('chatbot',)
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f069082-f073-62c0-8005-7b2f410d031d'}}


打印可以发现包括human，tool，assistant的消息

In [13]:
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values['messages']):
        print(state.values["messages"][-1])
        print("-" * 80)

Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
content='That’s a great idea! Building an autonomous agent with **LangGraph** is a fantastic way to explore its capabilities. You can create agents that **plan, execute tasks, use tools, and even collaborate** with other agents—all while maintaining state and supporting human oversight.\n\nHere’s a quick **example idea** to get you started:\n\n---\n\n### 🤖 **Autonomous Research Agent**\nThis agent takes a research question (e.g., *"What are the latest advancements in fusion energy?"*), breaks it down into steps, searches the web, summarizes findings, and compiles a report.\n\n#### Key Components:\n- **LLM Node**: For reasoning and planning.\n- **Search Tool**: To fetch up-to-date information.\n- **State Graph**: To manage flow between steps (plan → search → summarize → report).\n- **Human-in-the-loop**: Optional approval before finalizing the report.\n\n---\n\n### 🔧 Simple LangGr

## 4. Load a state from a moment-in-time¶

The checkpoint's to_replay.config contains a checkpoint_id timestamp. Providing this checkpoint_id value tells LangGraph's checkpointer to load the state from that moment in time.

In [14]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That’s a fantastic idea! Building an **autonomous agent** with LangGraph is a great way to explore its full potential. You can create agents that **plan, execute, adapt, and even collaborate** with other agents to achieve complex goals.

Here’s a quick roadmap to help you get started:

---

### 🚀 Step-by-Step: Build an Autonomous Agent with LangGraph

#### 1. **Define the Goal**
Pick a use case:
- A **research agent** that searches the web, summarizes articles, and compiles reports.
- A **personal task assistant** that manages your to-dos, schedules meetings, and sends emails.
- A **customer support bot** that routes issues and escalates when needed.

#### 2. **Design the Workflow (Graph)**
Use a **stateful graph** where each node is an action:
- `planning` → `